In [2]:
import numpy as np
from scipy import spatial 
import faiss
from time import time
import matplotlib.pyplot as plt
from collections import defaultdict

## Helper Function

In [3]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [4]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [5]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [6]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [7]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells and add the following results to the report:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [4]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k=10
dim = index_vectors.shape[1]

In [12]:
start = time()
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)
print(f"Exhaustive search took {time() - start:.2f} seconds.")

Exhaustive search took 4.06 seconds.


In [9]:
start = time()
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)
print(f"Building the LSH index took {time() - start:.2f} seconds.")

Building the LSH index took 0.62 seconds.


In [10]:
start = time()
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)
print(f"LSH search over query vectors took {time() - start:.2f} seconds.")

LSH search over query vectors took 0.13 seconds.


In [11]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.138


In [13]:
index_hnsw = faiss.IndexHNSWFlat(dim, 32)
index_hnsw.add(index_vectors)

start = time()
faiss_hnsw_ann = faiss_search(query_vectors, index_hnsw, k)
print(f"HNSW search over query vectors took {time() - start:.2f} seconds.")

print(f"recall@10 for faiss_hnsw_index: {compute_recall_at_k(gt_nn, faiss_hnsw_ann, k)}")

HNSW search over query vectors took 0.01 seconds.
recall@10 for faiss_hnsw_index: 0.859


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.8.

The last three bullets should also appear in the report.
You are allowed to add as many helper functions as you need. You cannot use faiss of scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:

    query_vectors = np.load('data/query_vectors2.npy')
    index_vectors = np.load('data/index_vectors2.npy')
or:

    query_vectors = np.load('data/query_vectors3.npy')
    index_vectors = np.load('data/index_vectors3.npy')
    
the aforementioned requirements should also be satisfied over these two query-index sets. No need to insert the results over these two to the report.

In [21]:
query_vectors2 = np.load('data/query_vectors2.npy')
index_vectors2 = np.load('data/index_vectors2.npy')

query_vectors3 = np.load('data/query_vectors3.npy')
index_vectors3 = np.load('data/index_vectors3.npy')

### Answers:

In [1]:
import numpy as np
from heapq import heappush, heappop
from collections import defaultdict

In [11]:
class HNSW_Node:
    def __init__(self, vector, level):
        self.vector = vector
        self.level = level
        self.neighbors = defaultdict(list)

In [17]:
class HNSW_Index:
    def __init__(self, dimension, max_level, num_neigh):
        self.dimension = dimension
        self.max_level = max_level
        self.num_neigh = num_neigh
        self.nodes = []
        self.start_node = None

    def distance(self, v1, v2):
        return np.linalg.norm(v1 - v2) # Euclidean distance

    def add_single(self, vector):
        new_node = HNSW_Node(vector, np.random.randint(0, self.max_level)) # random level for the new node

        if not self.nodes: # if the index is empty
            self.entry_point = new_node
            self.nodes.append(new_node)
        
        else:
            entry_point = self.entry_point
            for level in range(self.max_level, new_node.level, -1): # from highest to node level
                print(entry_point.neighbors[level])
                entry_point = min(entry_point.neighbors[level], key=lambda n: self.distance(vector, n.vector))

            for level in range(min(new_node.level, self.max_level)):
                candidates = self.search_layer(vector, entry_point, level, self.num_neigh)
                for candidate in candidates:
                    candidate.neighbors[level].append(new_node)
                    new_node.neighbors[level].append(candidate)

            self.nodes.append(new_node)
            if new_node.level > self.entry_point.level:
                self.entry_point = new_node

    def add(self, vectors):
        for vector in vectors:
            self.add_single(vector)

    def search_layer(self, query, entry_point, level, entry_factor):
        """ 
        Search for the nearest neighbors in the specified layer using the entry_point as the starting node.
        """
        visited = set()
        candidates = []
        best_candidates = []

        # init heap with (entry point, dist to query):
        heappush(candidates, (self.distance(query, entry_point.vector), entry_point))
        
        while candidates:
            dist, current_node = heappop(candidates)
            if current_node in visited:
                continue
            visited.add(current_node)

            if len(best_candidates) < entry_factor or dist < best_candidates[0][0]:
                heappush(best_candidates, (-dist, current_node))

                # keep the best_candidates list of size entry_factor:
                if len(best_candidates) > entry_factor:
                    heappop(best_candidates)

                for neighbor in current_node.neighbors[level]:
                    if neighbor not in visited:
                        heappush(candidates, (self.distance(query, neighbor.vector), neighbor))

        return [node for _, node in best_candidates]

    def search(self, query, k):
        entry_factor = max(k, self.ef_construction)
        entry_point = self.entry_point

        for level in range(self.max_level, 0, -1):
            entry_point = min(entry_point.neighbors[level], key=lambda n: self.distance(query, n.vector))

        candidates = self.search_layer(query, entry_point, 0, entry_factor)
        return sorted(candidates, key=lambda n: self.distance(query, n.vector))[:k]

In [19]:
hnsw = HNSW_Index(dimension=128, max_level=16, num_neigh=200)

# Add index vectors to the graph
hnsw.add(index_vectors)



[]


ValueError: min() iterable argument is empty

In [ ]:
# Search for the nearest neighbors of the query vectors
ann = []
for query in query_vectors:
    ann.append([node.vector for node in hnsw.search(query, k)])

print(f"recall@10 for hnsw_index: {compute_recall_at_k(gt_nn, np.array(ann), k)}")

In [ ]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.

def custom_indexing_algorithm(index_vectors, dim):
    """
    This function builds an index from scratch.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors.
    Returns:
        An index.
    """
    return 


def custom_index_search(query_vectors, index, k):
    """
    This function searches over the custom index.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors.
        index: The custom index.
        k: The number of nearest neighbors to retrieve.
    """
    return
    

In [ ]:
# Add hyperparameters here (if needed)

In [ ]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim)

In [ ]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

In [ ]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")